# Hopfield Net

In [1]:
import numpy as np

In [2]:
# Implementation for asynchronous Hopfield neural network
# Note: memory capacity ≃ 0.14*nodes
class Hopfield_Neural_Network:
    def __init__(self,nodes,iterations=100,weights=None):
        self.nodes = nodes
        self.iterations = iterations
        try:
            if weights == None:
                self.weights = np.zeros((nodes,nodes))
        except ValueError:
            self.weights = weights
    
    def store(self,input):
        dW = np.outer(input.transpose(),input)
        np.fill_diagonal(dW,0)
        self.weights += dW
        
    def recall(self,input):
        update_sequence = np.random.choice(self.nodes, self.iterations)
        for node in update_sequence:
            input[node] = np.sign(np.inner(input,self.weights[:,node]))
        return input

In [3]:
hnn = Hopfield_Neural_Network(10)

# Data

### Matrices

In [25]:
# TODO:
# need to convert matrices from 14*28 to 6*12 i.e. only the channels that can actually hold values

In [4]:
import re

In [5]:
def matrix_to_array(filename):
    f = open("/home/markus/dep/dep_matrices/front_back.dep","r")
    matrix = f.read()
    f.close()
    flattened = re.sub(",NEW_ROW", "", matrix).strip(",")
    flattened = re.split(",",flattened)
    flattened = np.array([float(weight) for weight in flattened])
    return flattened

In [6]:
# Front back
filename = "/home/markus/dep/dep_matrices/front_back.dep"
fb_matrix = matrix_to_array(filename)

In [7]:
# Front side
filename = "/home/markus/dep/dep_matrices/front_side.dep"
fs_matrix = matrix_to_array(filename)

In [8]:
# Side down
filename = "/home/markus/dep/dep_matrices/side_down.dep"
sd_matrix = matrix_to_array(filename)

### Transition points

In [9]:
behaviors = ["fb","fs","sd"]
transitions = [("fb","fs"), ("fb","sd"),("fs","fb"),("fs","sd"),("sd","fb"),("sd","fb")]
transition_muscle_2 = [[-1.5,-1.5,-1.5,-1.5,-1.5,-1.5],[1,1,1,1,1,1]] # [pos,...],[direction,..] -- direction = 1 -> up, direction = 0 -> down

In [10]:
# need to get positions and velocities for fb, fs and sd at -1.5 going up

In [11]:
import pickle

In [12]:
fb = pickle.load(open("/home/markus/dep/dep_data/bases/fb.pickle","rb"))
fs = pickle.load(open("/home/markus/dep/dep_data/bases/fs.pickle","rb"))
sd = pickle.load(open("/home/markus/dep/dep_data/bases/sd.pickle", "rb"))

In [13]:
# obtained from plot, time indices that meat transition_muscle_2 condition
fb_t = 124
fs_t = 126
sd_t = 117

In [14]:
# fb data
fb_pos = fb[0][fb_t]
fb_vel = fb[1][fb_t]
# fs data
fs_pos = fs[0][fs_t]
fs_vel = fs[1][fs_t]
# sd data
sd_pos = sd[0][sd_t]
sd_vel = sd[1][sd_t]

# Encoding

In [ ]:
# TODO:
# - need to create encoding for matrices, motor pos, motor vel, and brain id
# - need to create decoder

In [17]:
# need to create encoder that maps float value to [-1,1] sequence
# use HTM SDR Scalar Encoding
# Input: Scalar
# Parameters: n - number of units, w - bits used to represent signal (width), b - buckets (i.e. resolution), 
#             min - minimum value of input (inclusive), max - maximum input value (inclusive)
class scalar_sdr:
    
    def __init__(self, b, w, min_, max_):
        self.b = b # must be int
        self.w = w # must be int
        self.min = min_ # must be float
        self.max = max_ # must be float
        self.n = b+w-1 # number of units for encoding
        
    def encode(self,input_):
        if input_ > self.max or input_ < self.min:
            raise ValueError("InputError: Input outside encoder range!")
        output = np.zeros(self.n)-1
        index = int((input_-self.min)/(self.max-self.min)*self.b)
        output[index:index+self.w] = 1
        return output

In [18]:
# Example
encoder = scalar_sdr(380,21,0.0,100.0)
output_ = encoder.encode(50)